In [1]:
#can use verilog internal random functions (and thus not fixing IN_WIDTH)

In [2]:
#uses MatrixAdd_#M_#N_#pvPV_SI_HRx#hrf_(NIR, IR)_(NIL, IL) - v1.2

In [3]:
import sys
import math
import random

In [4]:
#M>=1
M = input('Enter No Of Vectors (M): ')

Enter No Of Vectors (M): 5


In [5]:
#N>=2
N = input('Enter No Of Elements Per Vectors (N): ')

Enter No Of Elements Per Vectors (N): 9


In [6]:
#PR>=1
PR = input('Enter No Of Parallel Vectors (empty for 1): ')

Enter No Of Parallel Vectors (empty for 1): 2


In [7]:
M = int(M)
if M<1:
    M=1
    
N = int(N)
if N<2:
    N=2
    
if not PR:
    PR=1
else:
    PR = int(PR)
    if PR>M:
        PR = M

In [8]:
#HRR>=2
HRR = input('Enter HRR Upper Limit (>=2, <={}, empty for 2, not all HRR values are acceptable): '.format(N))

Enter HRR Upper Limit (>=2, <=9, empty for 2, not all HRR values are acceptable): 6


In [9]:
if not HRR:
    HRR = 2
else:
    HRR = int(HRR)
    if HRR<2:
        HRR = 2
    if HRR>N:
        HRR = N

NAdd = math.ceil(N/HRR);
#print("NAdd: {}".format(NAdd))

while math.ceil(N/NAdd) != HRR:
    HRR -= 1
print("HRR calculated value: {}".format(HRR))

HRR calculated value: 5


In [10]:
#if IL is enabled, can generate extra random data between #hrr times (to show they are ignored correctly)
Enable_Extra_Data = input('Enable Extra Data Simulation Mode? (empty or 1 to enable, 0 to disable): ')

Enable Extra Data Simulation Mode? (empty or 1 to enable, 0 to disable): 


In [11]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH (empty for 10): ')

Enter IN_WIDTH (empty for 10): 18


In [12]:
Enable_Input_Registers = input('Enable Input Registers? (empty or 0 to disable, 1 to enable): ')

Enable Input Registers? (empty or 0 to disable, 1 to enable): 1


In [13]:
Enable_Input_Latch = input('Enable Input Latch? (empty or 0 to disable, 1 to enable): ')

Enable Input Latch? (empty or 0 to disable, 1 to enable): 1


In [14]:
if not Enable_Extra_Data:
    Enable_Extra_Data = 1
else:
    Enable_Extra_Data = int(Enable_Extra_Data)

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

if not Enable_Input_Registers:
    Enable_Input_Registers=0
else:
    Enable_Input_Registers = int(Enable_Input_Registers)

if not Enable_Input_Latch:
    Enable_Input_Latch=0
else:
    Enable_Input_Latch = int(Enable_Input_Latch)

In [15]:
ModuleName="MatrixAdd_{}_{}_{}PV_SI_HRx{}_".format(M, N, PR, HRR)
if Enable_Input_Registers<=0:
    ModuleName+="N"
ModuleName+="IR_"
if Enable_Input_Latch<=0:
    ModuleName+="N"
ModuleName+="IL"
UUTModuleName = ModuleName
ModuleName += "_test_IW{}".format(IN_WIDTH)

In [16]:
__Print_To_File = 0

if __Print_To_File<=0:
    of = sys.stdout
else:
    of = open("./"+ModuleName+".v", 'w+')

In [17]:
print("`timescale 1ns / 500ps\n", file=of)
print("module "+ModuleName+";", file=of)
print("//IN_WIDTH fixed to {}".format(IN_WIDTH), file=of)
print(file=of)

`timescale 1ns / 500ps

module MatrixAdd_5_9_2PV_SI_HRx5_IR_IL_test_IW18;
//IN_WIDTH fixed to 18



In [18]:
print("reg clk, reset, enable;", file=of)
print("wire readyForNewVectorStart;", file=of)
lghrr=math.ceil(math.log2(HRR))
if lghrr==1:
    print("wire vectorInSeries;", file=of)
elif lghrr>1:
    print("wire [{}:0] vectorInSeries;".format(lghrr-1), file=of)
RS=math.ceil(M/PR)
lgRS=math.ceil(math.log2(RS))
if lgRS<=1:
    print("wire vectorSetInNo;", file=of)
else:
    print("wire [{}:0] vectorSetInNo;".format(lgRS-1), file=of)
print("reg inReady;", file=of)
print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
for j in range(PR):
    for i in range(N):
        print("A{}V{}".format(i, j), end='', file=of)
        if j==(PR-1) and i==(N-1):
            print(";", file=of)
        elif i!=(N-1):
            print(", ", end='', file=of)            
        else:
            print(",", file=of)
#     print(file=of)
print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
for j in range(PR):
    for i in range(N):
        print("B{}V{}".format(i, j), end='', file=of)
        if j==(PR-1) and i==(N-1):
            print(";", file=of)
        elif i!=(N-1):
            print(", ", end='', file=of)            
        else:
            print(",", file=of)
#     print(file=of)
ar=RS*PR-M
ME = PR-ar-1
if ME==0:
    st1="V0"
else:
    st1="V0toV{}".format(ME)
print("wire ", end='', file=of)
NE = N - NAdd*(HRR-1)
while NE<=0:
    NE += NAdd
if NE==1:
    print(st1, end='', file=of)
    print("S0outReady;", file=of)
else:
    print(st1, end='', file=of)
    print("S0toS{}outReady;".format(NE-1), file=of)
print("wire ", end='', file=of)
if NE==NAdd:
    print("VSNoutReady1; //not used", file=of)
elif NE==NAdd-1:
    print(st1, end='', file=of)
    print("S{}outReady;".format(NE), file=of)
else:
    print(st1, end='', file=of)
    print("S{}toS{}outReady;".format(NE, NAdd-1), file=of)
if ar==0:
    print("wire VNoutReady1; //not used", file=of)
    print("wire VNoutReady2; //not used", file=of)
else:
    if ME==(PR-2):
        st2="V{}".format(PR-1)
    else:
        st2="V{}toV{}".format(ME+1, PR-1)
    print("wire ", end='', file=of)
    if NE==1:
        print(st2, end='', file=of)
        print("S0outReady;", file=of)
    else:
        print(st2, end='', file=of)
        print("S0toS{}outReady;".format(NE-1), file=of)
    print("wire ", end='', file=of)
    if NE==NAdd:
        print("VSNoutReady2; //not used", file=of)
    elif NE==NAdd-1:
        print(st2, end='', file=of)
        print("S{}outReady;".format(NE), file=of)
    else:
        print(st2, end='', file=of)
        print("S{}toS{}outReady;".format(NE, NAdd-1), file=of)
if lghrr==1:
    print("wire vectorOutSeries;", file=of)
elif lghrr>1:
    print("wire [{}:0] vectorOutSeries;".format(lghrr-1), file=of)
if lgRS<=1:
    print("wire vectorSetOutNo;", file=of)
else:
    print("wire [{}:0] vectorSetOutNo;".format(lgRS-1), file=of)
print("wire signed [{}:0] ".format(IN_WIDTH), end='', file=of)
for j in range(PR):
    for i in range(NAdd):
        print("S{}V{}".format(i, j), end='', file=of)
        if j==(PR-1) and i==(NAdd-1):
            print(";", file=of)
        elif i!=(NAdd-1):
            print(", ", end='', file=of)            
        else:
            print(",", file=of)
#     print(file=of)
print("wire ", end='', file=of)
if NE==1:
    print(st1, end='', file=of)
    print("S0earlyOutReady;", file=of)
else:
    print(st1, end='', file=of)
    print("S0toS{}earlyOutReady;".format(NE-1), file=of)
print("wire ", end='', file=of)
if NE==NAdd:
    print("CSNearlyOutReady1; //not used", file=of)
elif NE==NAdd-1:
    print(st1, end='', file=of)
    print("S{}earlyOutReady;".format(NE), file=of)
else:
    print(st1, end='', file=of)
    print("S{}toS{}earlyOutReady;".format(NE, NAdd-1), file=of)
if ar==0:
    print("wire CNearlyOutReady1; //not used", file=of)
    print("wire CNearlyOutReady2; //not used", file=of)
else:
    print("wire ", end='', file=of)
    if NE==1:
        print(st2, end='', file=of)
        print("S0earlyOutReady;", file=of)
    else:
        print(st2, end='', file=of)
        print("S0toS{}earlyOutReady;".format(NE-1), file=of)
    print("wire ", end='', file=of)
    if NE==NAdd:
        print("CSNearlyOutReady2; //not used", file=of)
    elif NE==NAdd-1:
        print(st2, end='', file=of)
        print("S{}earlyOutReady;".format(NE), file=of)
    else:
        print(st2, end='', file=of)
        print("S{}toS{}earlyOutReady;".format(NE, NAdd-1), file=of)

reg clk, reset, enable;
wire readyForNewVectorStart;
wire [2:0] vectorInSeries;
wire [1:0] vectorSetInNo;
reg inReady;
reg signed [17:0] A0V0, A1V0, A2V0, A3V0, A4V0, A5V0, A6V0, A7V0, A8V0,
A0V1, A1V1, A2V1, A3V1, A4V1, A5V1, A6V1, A7V1, A8V1;
reg signed [17:0] B0V0, B1V0, B2V0, B3V0, B4V0, B5V0, B6V0, B7V0, B8V0,
B0V1, B1V1, B2V1, B3V1, B4V1, B5V1, B6V1, B7V1, B8V1;
wire V0S0outReady;
wire V0S1outReady;
wire V1S0outReady;
wire V1S1outReady;
wire [2:0] vectorOutSeries;
wire [1:0] vectorSetOutNo;
wire signed [18:0] S0V0, S1V0,
S0V1, S1V1;
wire V0S0earlyOutReady;
wire V0S1earlyOutReady;
wire V1S0earlyOutReady;
wire V1S1earlyOutReady;


In [19]:
print(UUTModuleName, file=of)
print("#({})".format(IN_WIDTH), file=of)
print("UUT(", file=of)

print("clk, reset, enable,", file=of)
print("readyForNewVectorStart,", file=of)
print("vectorInSeries,", file=of)
print("vectorSetInNo,", file=of)
print("inReady,", file=of)
for j in range(PR):
    for i in range(N):
        print("A{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
for j in range(PR):
    for i in range(N):
        print("B{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
if NE==1:
    print(st1, end='', file=of)
    print("S0outReady,", file=of)
else:
    print(st1, end='', file=of)
    print("S0toS{}outReady,".format(NE-1), file=of)
if NE==NAdd:
    print("VSNoutReady1, //not used", file=of)
elif NE==NAdd-1:
    print(st1, end='', file=of)
    print("S{}outReady,".format(NE), file=of)
else:
    print(st1, end='', file=of)
    print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
if ar==0:
    print("VNoutReady1, //not used", file=of)
    print("VNoutReady2, //not used", file=of)
else:
    if NE==1:
        print(st2, end='', file=of)
        print("S0outReady,", file=of)
    else:
        print(st2, end='', file=of)
        print("S0toS{}outReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("VSNoutReady2, //not used", file=of)
    elif NE==NAdd-1:
        print(st2, end='', file=of)
        print("S{}outReady,".format(NE), file=of)
    else:
        print(st2, end='', file=of)
        print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
print("vectorOutSeries,", file=of)
print("vectorSetOutNo,", file=of)
for j in range(PR):
    for i in range(NAdd):
        print("S{}V{}, ".format(i, j), end='', file=of)
    print(file=of)
if NE==1:
    print(st1, end='', file=of)
    print("S0earlyOutReady,", file=of)
else:
    print(st1, end='', file=of)
    print("S0toS{}earlyOutReady,".format(NE-1), file=of)
if NE==NAdd:
    print("CSNearlyOutReady1, //not used", file=of)
elif NE==NAdd-1:
    print(st1, end='', file=of)
    print("S{}earlyOutReady,".format(NE), file=of)
else:
    print(st1, end='', file=of)
    print("S{}toS{}earlyOutReady,".format(NE, NAdd-1), file=of)
if ar==0:
    print("CNearlyOutReady1, //not used", file=of)
    print("CNearlyOutReady2 //not used", file=of)
else:
    if NE==1:
        print(st2, end='', file=of)
        print("S0earlyOutReady,", file=of)
    else:
        print(st2, end='', file=of)
        print("S0toS{}earlyOutReady,".format(NE-1), file=of)
    if NE==NAdd:
        print("CSNearlyOutReady2 //not used", file=of)
    elif NE==NAdd-1:
        print(st2, end='', file=of)
        print("S{}earlyOutReady".format(NE), file=of)
    else:
        print(st2, end='', file=of)
        print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
print(");\n", file=of)

MatrixAdd_5_9_2PV_SI_HRx5_IR_IL
#(18)
UUT(
clk, reset, enable,
readyForNewVectorStart,
vectorInSeries,
vectorSetInNo,
inReady,
A0V0, A1V0, A2V0, A3V0, A4V0, A5V0, A6V0, A7V0, A8V0, 
A0V1, A1V1, A2V1, A3V1, A4V1, A5V1, A6V1, A7V1, A8V1, 
B0V0, B1V0, B2V0, B3V0, B4V0, B5V0, B6V0, B7V0, B8V0, 
B0V1, B1V1, B2V1, B3V1, B4V1, B5V1, B6V1, B7V1, B8V1, 
V0S0outReady,
V0S1outReady,
V1S0outReady,
V1S1outReady,
vectorOutSeries,
vectorSetOutNo,
S0V0, S1V0, 
S0V1, S1V1, 
V0S0earlyOutReady,
V0S1earlyOutReady,
V1S0earlyOutReady,
V1S1earlyOutReady
);



In [20]:
InputRangeStart = -2**(IN_WIDTH-1)
InputRangeEnd = 2**(IN_WIDTH-1)
matrix_A = [[random.randrange(InputRangeStart, InputRangeEnd) for nn in range(N)] for jj in range(M)]
matrix_B = [[random.randrange(InputRangeStart, InputRangeEnd) for nn in range(N)] for jj in range(M)]

In [21]:
print(matrix_A)
print(matrix_B)

[[-41205, -17172, 96213, -15770, 79229, 62822, 50234, 78762, 58951], [-114770, 10578, 57153, 66269, 75709, 102282, 46260, 17846, -100773], [111237, -62886, -21922, -126157, 73955, 105844, -57725, -20150, -80890], [-55565, -17574, -33325, -16367, -72308, 72983, 62284, 64019, 22212], [-85909, -14196, 18222, -113081, 19693, -29374, 124488, -47833, -103347]]
[[23482, -40665, 38087, 49545, -41807, 70095, 116046, 25487, -124354], [32078, 50574, 98156, 115230, 6707, -4109, 113703, -11368, 18034], [69807, 47582, -56271, 43774, -42062, 40591, -92384, -21381, 32699], [-52823, 124013, -28381, 7308, 84823, -51893, -27268, -12673, -122055], [50858, 618, 24842, -117054, 34763, 38054, 46920, 35392, -12726]]


In [22]:
matrix_Res = [[0 for nn in range(N)] for jj in range(M)]
for jj in range(M):
    for nn in range(N):
        matrix_Res[jj][nn] = matrix_A[jj][nn]+matrix_B[jj][nn]
print(matrix_Res)

[[-17723, -57837, 134300, 33775, 37422, 132917, 166280, 104249, -65403], [-82692, 61152, 155309, 181499, 82416, 98173, 159963, 6478, -82739], [181044, -15304, -78193, -82383, 31893, 146435, -150109, -41531, -48191], [-108388, 106439, -61706, -9059, 12515, 21090, 35016, 51346, -99843], [-35051, -13578, 43064, -230135, 54456, 8680, 171408, -12441, -116073]]


In [23]:
print("initial begin", file=of)
print("\tclk = 1;", file=of)
print("\treset <= 0;", file=of)
print("\tenable <= 1;", file=of)
print("\tinReady <= 1;", file=of)

FirstInput = True
for j in range(RS):
    if FirstInput:
        FirstInput = False
    elif Enable_Extra_Data and Enable_Input_Latch:
        print("\t#1;", file=of)
    else:
        print("\t#{};".format(HRR), file=of)
        
    #print(file=of)
    print("\t//R", end='', file=of)
    for jj in range(j*PR, (j+1)*PR):
        if jj<M:
            print(" {}".format(jj), end='', file=of)
    print(file=of)
    
    jc=0
    for jj in range(j*PR, (j+1)*PR):
        if jj<M:
            for nn in range(N):
                print("\tA{}V{} <= {};".format(nn, jc, matrix_A[jj][nn]), file=of)
                cA = matrix_A[jj][nn]
                if cA >= 0:
                    print("\tA{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cA), file=of)
                else:
                    print("\tA{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cA)), file=of)                        
            jc += 1
        else:
            break
    
    jc=0
    for jj in range(j*PR, (j+1)*PR):
        if jj<M:
            for nn in range(N):
                cB = matrix_B[jj][nn]
                if cB >= 0:
                    print("\tB{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cB), file=of)
                else:
                    print("\tB{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cB)), file=of)                        
            jc += 1
        else:
            break

    if Enable_Extra_Data and Enable_Input_Latch:
        for h in range(HRR-1):
            print("\t#1;", file=of)
            print("\t//Extra Data:", file=of)
            jc=0
            for jj in range(j*PR, (j+1)*PR):
                if jj<M:
                    for nn in range(N):
                        cR = random.randrange(InputRangeStart, InputRangeEnd)
                        if cR >= 0:
                            print("\tA{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                        else:
                            print("\tA{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)
                    jc += 1
                else:
                    break
            jc=0
            for jj in range(j*PR, (j+1)*PR):
                if jj<M:
                    for nn in range(N):
                        cR = random.randrange(InputRangeStart, InputRangeEnd)
                        if cR >= 0:
                            print("\tB{}V{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                        else:
                            print("\tB{}V{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)
                    jc += 1
                else:
                    break

print("end", file=of)
print(file=of)

initial begin
	clk = 1;
	reset <= 0;
	enable <= 1;
	inReady <= 1;
	//R 0 1
	A0V0 <= -41205;
	A0V0 <= -18'd41205;
	A1V0 <= -17172;
	A1V0 <= -18'd17172;
	A2V0 <= 96213;
	A2V0 <= 18'd96213;
	A3V0 <= -15770;
	A3V0 <= -18'd15770;
	A4V0 <= 79229;
	A4V0 <= 18'd79229;
	A5V0 <= 62822;
	A5V0 <= 18'd62822;
	A6V0 <= 50234;
	A6V0 <= 18'd50234;
	A7V0 <= 78762;
	A7V0 <= 18'd78762;
	A8V0 <= 58951;
	A8V0 <= 18'd58951;
	A0V1 <= -114770;
	A0V1 <= -18'd114770;
	A1V1 <= 10578;
	A1V1 <= 18'd10578;
	A2V1 <= 57153;
	A2V1 <= 18'd57153;
	A3V1 <= 66269;
	A3V1 <= 18'd66269;
	A4V1 <= 75709;
	A4V1 <= 18'd75709;
	A5V1 <= 102282;
	A5V1 <= 18'd102282;
	A6V1 <= 46260;
	A6V1 <= 18'd46260;
	A7V1 <= 17846;
	A7V1 <= 18'd17846;
	A8V1 <= -100773;
	A8V1 <= -18'd100773;
	B0V0 <= 18'd23482;
	B1V0 <= -18'd40665;
	B2V0 <= 18'd38087;
	B3V0 <= 18'd49545;
	B4V0 <= -18'd41807;
	B5V0 <= 18'd70095;
	B6V0 <= 18'd116046;
	B7V0 <= 18'd25487;
	B8V0 <= -18'd124354;
	B0V1 <= 18'd32078;
	B1V1 <= 18'd50574;
	B2V1 <= 18'd98156;
	B3V1 <= 18'd115

In [24]:
print("always #0.5 clk = ~ clk;", file=of)
print(file=of)

initial_delay_p = 0
print("//initial delay:", file=of)
if Enable_Input_Registers<=0:
    print("//NIR: ", end='', file=of)
    print("0", file=of)
else:
    print("//IR: ", end='', file=of)
    print("1", file=of)
    initial_delay_p += 1
print("//Registered2to1Adder: 1", file=of)
initial_delay_p += 1
print("parameter initial_delay = {};".format(initial_delay_p), file=of)
print(file=of)

print("integer outfile;", file=of)
print(file=of)

print("task Print_Parameters_Close;", file=of)
print("\tbegin", file=of)
print("\t$fdisplay(outfile, \"\\nParameters' values:\");", file=of)
print("\t$fdisplay(outfile, \"IN_WIDTH : fixed to {}\");".format(IN_WIDTH), file=of)
print("\t$fclose(outfile);", file=of)
print("\tend", file=of)
print("endtask", file=of)
print(file=of)

print("task Simulation_Fail;", file=of)
print("\tbegin", file=of)
print("\t$fdisplay(outfile, \"Failed\");", file=of)
print("\t$fdisplay(outfile, \"Error at %t\", $realtime);", file=of)
print("\tPrint_Parameters_Close();", file=of)
print("\t$display(\"Failed\");", file=of)
print("\tend", file=of)
print("endtask", file=of)
print(file=of)

always #0.5 clk = ~ clk;

//initial delay:
//IR: 1
//Registered2to1Adder: 1
parameter initial_delay = 2;

integer outfile;

task Print_Parameters_Close;
	begin
	$fdisplay(outfile, "\nParameters' values:");
	$fdisplay(outfile, "IN_WIDTH : fixed to 18");
	$fclose(outfile);
	end
endtask

task Simulation_Fail;
	begin
	$fdisplay(outfile, "Failed");
	$fdisplay(outfile, "Error at %t", $realtime);
	Print_Parameters_Close();
	$display("Failed");
	end
endtask



In [25]:
print("initial begin", file=of)
print("$timeformat (-9, 1, \" ns\", 0);", file=of)
print("outfile = $fopen(\"_{}_Simulation_Result.txt\", \"w\");".format(IN_WIDTH), file=of)
print("#initial_delay;", file=of)
print("#0.5;", file=of)

initial begin
$timeformat (-9, 1, " ns", 0);
outfile = $fopen("_18_Simulation_Result.txt", "w");
#initial_delay;
#0.5;


In [28]:
FirstOutputSeries = True
for j in range(RS):
    for h in range(HRR):
        if FirstOutputSeries:
            FirstOutputSeries = False
        else:
            print("#1;", file=of)
        jc=0
        for jj in range(j*PR, (j+1)*PR):
            if jj<M:
                print("if(", end='', file=of)
                nc=0
                FirstOutput = True
                for nn in range(h*NAdd, (h+1)*NAdd):
                    if nn<N:
                        if FirstOutput:
                            FirstOutput = False
                        else:
                            print(" || ", end='', file=of)
                        cRes = matrix_Res[jj][nn]
                        if cRes >= 0:
                            print("S{}V{}!={}'d{}".format(nc, jc, IN_WIDTH+1, cRes), end='', file=of)
                        else:
                            print("S{}V{}!=-{}'d{}".format(nc, jc, IN_WIDTH+1, abs(cRes)), end='', file=of)
                        nc += 1
                    else:
                        break
                print(") begin", file=of)
                print("\tSimulation_Fail();", file=of)
                print("\t$finish;", file=of)
                print("end", file=of)
                jc += 1
            else:
                break

if(S0V0!=-19'd17723 || S1V0!=-19'd57837) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=-19'd82692 || S1V1!=19'd61152) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=19'd134300 || S1V0!=19'd33775) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=19'd155309 || S1V1!=19'd181499) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=19'd37422 || S1V0!=19'd132917) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=19'd82416 || S1V1!=19'd98173) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=19'd166280 || S1V0!=19'd104249) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=19'd159963 || S1V1!=19'd6478) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=-19'd65403) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=-19'd82739) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=19'd181044 || S1V0!=-19'd15304) begin
	Simulation_Fail();
	$finish;
end
if(S0V1!=-19'd108388 || S1V1!=19'd106439) begin
	Simulation_Fail();
	$finish;
end
#1;
if(S0V0!=-19'd78193 || S1V0!=-19'd82383) begin
	Si

In [29]:
print("#0.5;", file=of)
print("$fdisplay(outfile, \"Succeeded\");", file=of)
print("$fdisplay(outfile, \"Simulation completed without error at %t\", $realtime);", file=of)
print("Print_Parameters_Close();", file=of)
print("$display(\"Succeeded\");", file=of)
print("$finish;", file=of)
print("end", file=of)
print(file=of)
print("endmodule", file=of)

#0.5;
$fdisplay(outfile, "Succeeded");
$fdisplay(outfile, "Simulation completed without error at %t", $realtime);
Print_Parameters_Close();
$display("Succeeded");
$finish;
end

endmodule


In [30]:
if __Print_To_File>0:
    of.close()